In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from unidecode import unidecode
from difflib import SequenceMatcher
import re
import requests
import joblib

# Processamento/tratamento de dados

## Leitura do arquivo inicial com as compras.

In [2]:
df = pd.read_csv('arquivos/Arquivo inicial _202202191053.csv')

Breve verificação da integridade dos dados do arquivo.

In [3]:
df.head(10)

,id_pedido,data_pedido,id_cliente,nome_cliente,bairro_pedido,cidade_pedido,cep_pedido,nome_produto,id_produto,qnt_itens_vendidos,preco_itens,tipo_pagamento,agente
0,54,2020-07-03 11:30:53.000,39,Barbara,aldeota,Fortaleza,60150060,cajuína,54,3,10.0,Pagar ao retirar na loja,Mozilla/5.0 (Linux; Android 9; motorola one zo...
1,54,2020-07-03 11:30:53.000,39,Barbara,aldeota,Fortaleza,60150060,água de coco,56,2,3.0,Pagar ao retirar na loja,Mozilla/5.0 (Linux; Android 9; motorola one zo...
2,54,2020-07-03 11:30:53.000,39,Barbara,aldeota,Fortaleza,60150060,couve,75,1,3.0,Pagar ao retirar na loja,Mozilla/5.0 (Linux; Android 9; motorola one zo...
3,54,2020-07-03 11:30:53.000,39,Barbara,aldeota,Fortaleza,60150060,couve,75,1,3.0,Pagar ao retirar na loja,Mozilla/5.0 (Linux; Android 9; motorola one zo...
4,54,2020-07-03 11:30:53.000,39,Barbara,aldeota,Fortaleza,60150060,doce de leite,77,1,12.0,Pagar ao retirar na loja,Mozilla/5.0 (Linux; Android 9; motorola one zo...
5,54,2020-07-03 11:30:53.000,39,Barbara,aldeota,Fortaleza,60150060,queijo coalho,94,2,25.0,Pagar ao retirar na loja,Mozilla/5.0 (Linux; Android 9; motorola one zo...
6,56,2020-07-03 11:32:27.000,38,Anna Karla,são gerardo,Fortaleza,60320105,arroz parboilizado,59,1,6.0,Cartão de Crédito,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
7,56,2020-07-03 11:32:27.000,38,Anna Karla,são gerardo,Fortaleza,60320105,banana maçã,60,1,6.0,Cartão de Crédito,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
8,56,2020-07-03 11:32:27.000,38,Anna Karla,são gerardo,Fortaleza,60320105,banana prata,61,2,5.0,Cartão de Crédito,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
9,56,2020-07-03 11:32:27.000,38,Anna Karla,são gerardo,Fortaleza,60320105,biscoito de castanha,63,2,3.5,Cartão de Crédito,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...


## Significado dos campos
Neste arquivo temos, em formato long, uma lista de pedidos realizados. A seguir encontra-se a descrição de cada coluna:

- id_pedido: código único que identifica cada pedido;
- data_pedido: data e hora em que os pedidos foram realizados;
- id_cliente: código único que identifica o cliente que fez o pedido;
- nome_cliente: nome do cliente que fez o pedido. Os nomes foram distorcidos para evitar identificação das pessoas;
- bairro_pedido: bairro onde deve serfeita a entrega do pedido;
- cidade_pedido: cidade onde deve ser feita a entrega do pedido;
- cep_pedido: CEP onde deve ser feita a entrega do pedido;
- nome_produto: nome do produto;
- id_produto: código de identificação do produto;
- qnt_itens_vendidos: quantidade de itens identificados por nome_produto e id_produto;
- preco_itens: preço unitário do item;
- tipo_pagamento: forma de pagamento;
- agente: plataforma utilizada pelo cliente no momento em que fez o pedido.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36996 entries, 0 to 36995
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id_pedido           36996 non-null  int64  
 1   data_pedido         36996 non-null  object 
 2   id_cliente          36996 non-null  int64  
 3   nome_cliente        36996 non-null  object 
 4   bairro_pedido       36505 non-null  object 
 5   cidade_pedido       36996 non-null  object 
 6   cep_pedido          36463 non-null  object 
 7   nome_produto        36996 non-null  object 
 8   id_produto          36996 non-null  int64  
 9   qnt_itens_vendidos  36996 non-null  int64  
 10  preco_itens         36996 non-null  float64
 11  tipo_pagamento      36996 non-null  object 
 12  agente              36996 non-null  object 
dtypes: float64(1), int64(4), object(8)
memory usage: 3.7+ MB


## Verificação de valores nulos

In [5]:
df.isnull().sum()

id_pedido               0
data_pedido             0
id_cliente              0
nome_cliente            0
bairro_pedido         491
cidade_pedido           0
cep_pedido            533
nome_produto            0
id_produto              0
qnt_itens_vendidos      0
preco_itens             0
tipo_pagamento          0
agente                  0
dtype: int64

É possível verificar que temos valores nulos nas colunas de bairro e CEP. Podemos utilizar o CEP para descobrir o bairro, mas o inverso não é possível. Vamos, portanto, inicialmente, remover as linhas sem CEP.<p>
O tratamento de bairros será feito mais à frente, portanto não excluiremos valores nulos nesta coluna neste momento.

In [6]:
ceps_nulos = df[df['cep_pedido'].isnull()]
df.drop(ceps_nulos.index,inplace=True)

## Padronização dos códigos e nomes dos produtos
Antes de qualquer coisa, para garantir os tratamentos que serão feitos mais à frente, vamos retirar caracteres maiúsculos e acentos dos nomes dos produtos.

In [7]:
df['nome_produto'] = df['nome_produto'].str.strip()
df['nome_produto'] = df['nome_produto'].str.lower()
df['nome_produto'] = df['nome_produto'].apply(lambda prod: unidecode(prod))

Nomes de produtos, como será possível verificar mais à frente, nos serão essenciais, portanto caso haja algum produto sem nome, deve ser removido. Acima já foi possível verificar que não há valores nulos na coluna de nomes de produtos, mas também precisamos procurar se temos espaços.

In [8]:
((df['nome_produto'].str.isspace()) | (df['nome_produto'] == '')).sum()

0

Não há espaços. Prossigamos.

Percebe-se que há uma certa inconsistência tanto nos nomes quanto nos códigos dos produtos. Perceba os valores distintos nos contadores a seguir.

In [9]:
len(df['nome_produto'].unique())

253

In [10]:
len(df['id_produto'].unique())

177

Mais adiante utilizaremos os nomes para realizar análises, já que tratam-se de informações muito mais amigáveis, porém neste momento podemos nos utilizar do código para perceber que existem nomes distintos para um mesmo produto, o que acabará causando a divisão da contabilização de totais destes produtos em grupos distintos, o que afetará a análise.<P>
Primeiramente vamos ver códigos de produto que estão sendo utilizados para mais de um nome de produto.

In [11]:
# Montando um dicionário onde as chaves são os ids de os valores são um set com os nomes de produtos.
prods = dict()
for idp in df['id_produto'].unique():
    nomes_id = df[df['id_produto'] == idp]['nome_produto'].unique()
    prods[idp] = set(nomes_id)

# Mostrando uma amostra de códidos de produtos que estão sendo utilizados para mais de um nome
i = int()
for id_p,nomes in prods.items():
    if len(nomes) != 1:
        i += 1
        if i > 5:
            break
        print(f"{id_p}: {nomes} - {len(nomes)} nomes distintos")

54: {'cajuina 1l', 'cajuina'} - 2 nomes distintos
56: {'agua de coco [ garrafa 300ml ]', 'agua de coco'} - 2 nomes distintos
77: {'doce de leite [ pote ]', 'doce de leite'} - 2 nomes distintos
94: {'queijo coalho 1kg', 'queijo coalho (1kg)', 'queijo coalho'} - 3 nomes distintos
59: {'arroz parboilizado organico - 1kg', 'arroz parboilizado'} - 2 nomes distintos


Vamos verificar diretamente no data set se realmente há esta ambiguidade para os códigos de produto. Peguemos o código 54 como exemplo.

In [12]:
df[df['id_produto'] == 54][['id_produto','nome_produto']].sample(5,random_state=2)

,id_produto,nome_produto
18988,54,cajuina
35674,54,cajuina 1l
16343,54,cajuina
20452,54,cajuina
25878,54,cajuina


É possível perceber que os nomes são muito parecidos. Optaremos, portanto, em padronizá-los adotando como nome a porção similar entre eles. Por exemplo, entre <i>cajuina 1l</i> e <i>cajuina</i> ficaremos com <i>cajuina</i>.

In [13]:
# Primeiramente vamos criar uma função que compara dois strings e retorna a porção similar entre eles.
def texto_igual(a,b):
    sm = SequenceMatcher(None,a,b).get_matching_blocks()
    str_sim = (a[sm[0][0]:sm[0][0] + sm[0][2]]).strip()
    return str_sim


# Tratamento dos nomes dos produtos, colocando os nomes de produtos definitivo no DataFrame.
nomes_def = dict() # Chave: código de produto, valor: nome definitivo atrelado ao código
nome_def = str() # Necessário para guardar o texto comum quanto há mais de dois nomes.

# Iterando entre os ids com produtos duplicados e selecionando o nome definitivo.
for id_p,nomes in prods.items():
    if len(nomes) == 1: # Só um nome. Não é preciso verificação.
        nomes_def[id_p] = nomes.pop()
            
    else: 
        for i,nome in enumerate(nomes): # É preciso comparar todas ocorrências. Há casos onde há mais de dois nomes.
            if i == 0: # O primeiro nome se torna a  base de comparação
                nome_def = nome
            else:
                nome_def = texto_igual(nome_def,nome)

        nomes_def[id_p] = nome_def
    
df['nome_produto'] = df.id_produto.map(nomes_def)

Verificando se a alteração surtiu efeito.

In [14]:
df[df['id_produto'] == 54][['id_produto','nome_produto']].sample(5,random_state=2)

,id_produto,nome_produto
18988,54,cajuina
35674,54,cajuina
16343,54,cajuina
20452,54,cajuina
25878,54,cajuina


Vamos fazer mais uma comparação da quantidade de valores únicos nos campos de id de produto e código de produto, Se tudo estiver íntegro, as quantidades deveriam estar iguais.

In [15]:
len(df['id_produto'].unique())

177

In [16]:
len(df['nome_produto'].unique())

176

Ficamos com a situação contrária. Há mais ids do que nomes de produto. Vamos descobrir quais são usando a mesma lógica utilizada anteriormente, porém invertendo os campos.

In [17]:
ids = dict()
for nome in df['nome_produto'].unique():
    ids_nome = df[df['nome_produto'] == nome]['id_produto'].unique()
    ids[nome] = set(ids_nome)

i = int()
for nome,ids_nome in ids.items():
    if len(ids_nome) != 1:
        i += 1
        if i > 5:
            break
        print(f"{nome}: {ids_nome} - {len(ids_nome)} ids distintos")

arroz: {200, 57} - 2 ids distintos


Como se trata de uma única ocorrência, vamos corrigir o valor manualmente.

In [18]:
df['id_produto'].replace({200:57},inplace=True)

Finalmente os campos estão íntegros e com redundâncias tratadas, conforme é possível ver nos contadores a seguir.

In [19]:
len(df['nome_produto'].unique())

176

In [20]:
len(df['id_produto'].unique())

176

Em passos posteriores, pecebemos que um código específico não estava sendo resolvido pelo tratamento automático devido à maneira como seus nomes foram cadastrados. <i><b>C</b>olorau</i>, <i><b>c</b>orante de urucum</i> e <i><b>c</b>orante de urucum [ 200 g ]</i>. O nome de produto resultante do processo automático estava sendo <b>c</b>. Por esta razão, este é o único produto que necessita de intervenção manual.

In [21]:
df['nome_produto'] = df['nome_produto'].replace({'c':'corante de urucum'})

O id do produto não nos tem mais serventia. Vamos excluí-lo.

In [22]:
df.drop('id_produto',axis=1,inplace=True)

## Tratamento e validação do conteúdo das colunas
É necessária análise e tratamento do conteúdo de todas as colunas para avaliar de qual forma seus conteúdos poderão ser utilizados.<p>

### id_cliente
Vamos verificar se algum id do cliente tem mais de um nome. Como o id do cliente será utilizado mais para frente, seu compartilhamento entre vários clientes estaria totalizando vários clientes como um só, o que pode afetar nossa análise.

In [23]:
ids_clientes = df['id_cliente'].unique()

for id_cliente in ids_clientes:
    nomes_id = df[df['id_cliente'] == id_cliente]['nome_cliente'].unique()
    if len(nomes_id) > 1:
        print(f"{id_cliente}: {nomes_id} - {len(nomes_id)} nomes")

0: ['Riviane ' 'Ticiana ' 'Stefanie' 'Karla Maria' 'Vanessa' 'Tatiany' 'José'
 'Celecina ' 'daniel kayo' 'silvia' 'MARIA JOSE ' 'Sheila' 'Rita '
 'Lorena' 'Margareth ' 'Mayra' 'Cinthia' 'samya' 'Indra' 'Virna ' 'Sônia'
 'Vicente' 'Tatiana ' 'Naggila' 'Rodrigo ' 'Juliana ' 'Marcos'
 'LUIZA ADRIANA' 'Rita' 'Rafaela' 'Samya' 'Margareth' 'claudene ' 'Naike'
 'Maria do Carmo ' 'Annya' 'Raíssa' 'Rosane' 'Pedro' 'Ercilia' 'jullyana '
 'Beatriz ' 'Rosilene ' 'Rebeca' 'Ana Cristina' 'Carlos ' 'Tatiany '
 'Daniel' 'Keyla' 'Paulo Marcelo' 'Amanda' 'Rayanne ' 'Samira'
 'Francisco Maxshwell dos Santos ' 'Talisson' 'Rosilene' 'Leonardo'
 'Paolla ' 'Gema Galgani Silveira' 'Maria Auxiliadora ' 'Lina'
 'Guilherme ' 'Gracas' 'Paolla' 'Indra ' 'CLAUDIA MARIA' 'Gabriel '
 'Nathan' 'Jessica' 'MARGARETH' 'Salete' 'Elizabete ' 'Jacinta Márcia'
 'Karine' 'João Paulo' 'Nathalia' 'Luz Elena ' 'Noandro '] - 78 nomes


Há 78 nomes compartilhando o id 0. Vamos criar 78 ids fictícios para cada um desses nomes, mas antes vamos verificar a quantidade de ids únicos para validar a alteração que será feita.

In [24]:
len(df['id_cliente'].unique())

1123

Vamos às alterações.

In [25]:
# Criando um dicionário com um id fictício para cada nome
fake_id = 9999
dic_nomes = dict()
nomes = df[df['id_cliente'] == 0]['nome_cliente'].unique()

for nome in nomes:
    dic_nomes[nome] = fake_id
    fake_id -= 1

# Transformando ids zerados em seus respectivos nomes
df['id_cliente'] = np.where(df['id_cliente'] == 0, df['nome_cliente'],df['id_cliente'])

# Substituindo nomes por id ficitício na coluna de ids
df['id_cliente'].replace(dic_nomes,inplace=True)

Vamos validar as alterações feitas verificando a quantidade de ids de cliente únicos. Deveríamos ter a quantidade original + 78 ficítícios - 1, que é o zero que foi eliminado.

In [26]:
len(df['id_cliente'].unique())

1200

### nome_cliente
A coluna com o nome do cliente não será utilizada, portanto podemos removê-la.

In [27]:
df.drop('nome_cliente',axis=1,inplace=True)

### agente
A coluna mostra a plataforma utilizada pelo cliente, porém de maneira pouco amigável. Vamos simplificá-la criando 8 classes:


In [28]:
def retorna_agente(agente):
    if 'android' in agente.lower():
        return 'android'
    elif 'ipad' in agente.lower():
        return 'ipad'
    elif 'iphone' in agente.lower():
        return 'iphone'
    elif 'macintosh' in agente.lower():
        return 'macintosh'
    elif 'windows' in agente.lower():
        return 'windows'
    elif 'linux' in agente.lower():
        return 'linux'
    elif 'x11' in agente.lower():
        return 'x11'
    return 'desconhecido'

df['agente'] = df['agente'].apply(retorna_agente)

### cidade_pedido e bairro_pedido
Como são informações manualmente preenchidas pelo cliente, podem conter erros de digitação ou diferenças devido a letras maiúsculas, minúsculas e acentos. Como mais a frente pretendemos fazer análises totalizando valores por bairro e cidade, é interessante que tenhamos estas informações padronizadas. Para tanto, nos utilizaremos de arquivo csv gerado com ***1.2 - Captura de bairros e cidades a partir do CEP.ipynb***.

In [29]:
df_ends = pd.read_csv('arquivos/Endereços dos CEPs.csv',dtype={'cep':'string'})
df_ends.head()

,cep,bairro,cidade
0,60150060,aldeota,fortaleza
1,60320105,sao gerardo,fortaleza
2,60713480,manoel satiro,fortaleza
3,60431075,rodolfo teofilo,fortaleza
4,60430660,rodolfo teofilo,fortaleza


É preciso deixar o CEP no DataFrame de compras no mesmo formato que acabou de ser lido no arquivo que foi guardado em <code>df_ends</code>.

In [30]:
df['cep_pedido'] = df['cep_pedido'].apply(lambda cep: re.sub('[^0-9]','',cep))
df['cep_pedido'] = df['cep_pedido'].apply(lambda cep: f"{int(cep):08d}")

Vamos primeiramente definir uma função que recebe CEP, bairro e endereço e retorna as informações presentes no arquivo correspondentes ao CEP informado. Bairro e endereço também devem ser recebidos para que possam ser retornados caso o CEP não esteja presente no arquivo. Tal lógica foi adotada para evitarmos de perder, no DataFrame, informação de cidade e bairro para casos onde foi informado um CEP inválido.<p>
Além disso, CEPs inválidos serão zerados.

In [31]:
df.reset_index(drop=True, inplace=True) # Necessário pois linhas foram "dropadas" e o índice não está mais contínuo.

def busca_end(cep,bairro,cidade):
    if cep in list(df_ends['cep']):
        bairro_lido = df_ends[df_ends['cep'] == cep]['bairro'].to_string(index=False)
        cidade_lida = df_ends[df_ends['cep'] == cep]['cidade'].to_string(index=False)
        return cep,bairro_lido,cidade_lida
    else:
        return '0'*8,bairro,cidade

ends_finais = df[['cep_pedido','bairro_pedido','cidade_pedido']].apply(
    lambda df: busca_end(df['cep_pedido'],df['bairro_pedido'],df['cidade_pedido']),axis=1)
    
df['cep_pedido'] = pd.DataFrame(list(ends_finais))[0]
df['bairro_pedido'] = pd.DataFrame(list(ends_finais))[1]
df['cidade_pedido'] = pd.DataFrame(list(ends_finais))[2]

Cidades e bairros de registros cujo CEP não é válido podem ter ficado com caracteres maiúsculos, espaços desnecessários ou acentos. Vamos retirá-los.

In [32]:
df['bairro_pedido'].fillna(' ',inplace=True)
df['bairro_pedido'] = df['bairro_pedido'].map(str)
df['bairro_pedido'] = df['bairro_pedido'].str.strip()
df['bairro_pedido'] = df['bairro_pedido'].str.lower()
df['bairro_pedido'] = df['bairro_pedido'].apply(lambda prod: unidecode(prod))

df['cidade_pedido'].fillna(' ',inplace=True)
df['cidade_pedido'] = df['cidade_pedido'].map(str)
df['cidade_pedido'] = df['cidade_pedido'].str.strip()
df['cidade_pedido'] = df['cidade_pedido'].str.lower()
df['cidade_pedido'] = df['cidade_pedido'].apply(lambda prod: unidecode(prod))

Vamos verificar a situação final deste campos.

In [33]:
df['cidade_pedido'].value_counts()

fortaleza    35570
caucaia        447
maracanau      171
aquiraz        125
osasco          57
fortim          44
eusebio         35
salvador         9
sao paulo        3
itaitinga        2
Name: cidade_pedido, dtype: int64

In [34]:
df['bairro_pedido'].value_counts()

meireles             4666
aldeota              2832
joaquim tavora       2744
benfica              2278
fatima               1950
                     ... 
parque leblon           2
pari                    1
parque santa rosa       1
republica               1
vila mariana            1
Name: bairro_pedido, Length: 132, dtype: int64

### data_pedido
A data do pedido está como 'object'. Vamos transformá-la em datetime e separar data e hora em duas colunas distintas.

In [35]:
df['hora_pedido'] = pd.to_datetime(df['data_pedido']).dt.time
df['data_pedido'] = pd.to_datetime(df['data_pedido']).dt.date

### Categoria do produto
Utilizando um modelo treinado em ***1.1 - Treinamento de modelo para classificação de categoria.ipynb***, vamos acrescentar uma coluna que informa a categoria do produto.

In [36]:
modelo_categ = joblib.load('arquivos/modelos/modelo_nb_classificacao_categorias.pkl')
tfidf_categ = joblib.load('arquivos/modelos/tfidf_classificacao_categorias.pkl')
prods_tfidf = tfidf_categ.transform(df['nome_produto'])
df['categoria'] = modelo_categ.predict(prods_tfidf)

### Unificação de produtos duplicados em um mesmo pedido
Perceba que o produto *couve* aparece duas vezes no pedido 54.

In [37]:
df.head()

,id_pedido,data_pedido,id_cliente,bairro_pedido,cidade_pedido,cep_pedido,nome_produto,qnt_itens_vendidos,preco_itens,tipo_pagamento,agente,hora_pedido,categoria
0,54,2020-07-03,39,aldeota,fortaleza,60150060,cajuina,3,10.0,Pagar ao retirar na loja,android,11:30:53,sucos refrescos e refrigerantes
1,54,2020-07-03,39,aldeota,fortaleza,60150060,agua de coco,2,3.0,Pagar ao retirar na loja,android,11:30:53,agua energeticos e chas
2,54,2020-07-03,39,aldeota,fortaleza,60150060,couve,1,3.0,Pagar ao retirar na loja,android,11:30:53,verduras
3,54,2020-07-03,39,aldeota,fortaleza,60150060,couve,1,3.0,Pagar ao retirar na loja,android,11:30:53,verduras
4,54,2020-07-03,39,aldeota,fortaleza,60150060,doce de leite,1,12.0,Pagar ao retirar na loja,android,11:30:53,doces e sobremesas


Vamos unificar essas linhas, somando a quantidade de produtos.

In [38]:
colunas = list(df.columns)
colunas.remove('qnt_itens_vendidos')

df = df.groupby(colunas).agg({'qnt_itens_vendidos':['sum']})
df.reset_index(inplace=True)
df.columns = [col[0] for col in df.columns]

### Criação de coluna de valor total dos produtos.
Por enquanto só temos uma coluna com o valor unitário. Vamos criar uma coluna com o total gasto com todos os itens.

In [39]:
# Melhorando o nome de algumas colunas
df.columns = [col if col != 'preco_itens' else 'vl_unitario' for col in list(df.columns)]
df.columns = [col if col != 'qnt_itens_vendidos' else 'qt_itens' for col in list(df.columns)]

# Criando a nova coluna
df['vl_total'] = (df['qt_itens'] * df['vl_unitario']).round(2)

Vamos dar uma olhada no DataFrame após todos os tratamentos realizados.

In [40]:
df.head()

,id_pedido,data_pedido,id_cliente,bairro_pedido,cidade_pedido,cep_pedido,nome_produto,vl_unitario,tipo_pagamento,agente,hora_pedido,categoria,qt_itens,vl_total
0,54,2020-07-03,39,aldeota,fortaleza,60150060,agua de coco,3.0,Pagar ao retirar na loja,android,11:30:53,agua energeticos e chas,2,6.0
1,54,2020-07-03,39,aldeota,fortaleza,60150060,cajuina,10.0,Pagar ao retirar na loja,android,11:30:53,sucos refrescos e refrigerantes,3,30.0
2,54,2020-07-03,39,aldeota,fortaleza,60150060,couve,3.0,Pagar ao retirar na loja,android,11:30:53,verduras,2,6.0
3,54,2020-07-03,39,aldeota,fortaleza,60150060,doce de leite,12.0,Pagar ao retirar na loja,android,11:30:53,doces e sobremesas,1,12.0
4,54,2020-07-03,39,aldeota,fortaleza,60150060,queijo coalho,25.0,Pagar ao retirar na loja,android,11:30:53,queijos,2,50.0


## Criação de DataFrame em formato wide.
Até então visualizamos os dados somente em formato long, com uma linha por produto, havendo, portanto, várias linhas por pedido. Para diversas análises que pretendemos fazer, é necessário termos os dados em formato wide, ou seja, havendo somente uma linha por produto.<p>
Criaremos, portanto, mais um DataFrame, onde cada produto será representado por variáveis dummy, havendo uma coluna por quantidade de cada produto e outra para o valor do item. Também acrescentaremos uma coluna com o valor total e outra com a quantidade total de itens da compra, que poderão ser úteis em algumas análises.

In [41]:
pedidos = dict()

for i in range(0,len(df)):
    linha = df.iloc[i]
    if linha['id_pedido'] not in pedidos.keys():
        pedidos[linha['id_pedido']] = {'data_pedido':linha['data_pedido'],
                                       'hora_pedido':linha['hora_pedido'],
                                       'id_cliente':linha['id_cliente'],
                                       'bairro_pedido':linha['bairro_pedido'],
                                       'cidade_pedido':linha['cidade_pedido'],
                                       'cep_pedido':linha['cep_pedido'],
                                       'tipo_pagamento':linha['tipo_pagamento'],
                                       'agente':linha['agente'],
                                       'i_' + linha['nome_produto']:True,
                                       'qt_'+linha['nome_produto']:linha['qt_itens'],
                                       'vl_tot_'+linha['nome_produto']:linha['vl_total'],
                                       'vl_final_pedido':linha['vl_total'],
                                       'qt_final_itens': linha['qt_itens']}
    else:
        pedidos[linha['id_pedido']]['vl_final_pedido'] += linha['vl_total']
        pedidos[linha['id_pedido']]['qt_final_itens'] += linha['qt_itens']
        pedidos[linha['id_pedido']]['i_' + linha['nome_produto']] = True
        pedidos[linha['id_pedido']]['qt_'+linha['nome_produto']] = linha['qt_itens']
        pedidos[linha['id_pedido']]['vl_tot_'+linha['nome_produto']] = linha['vl_total']

df_ped = pd.DataFrame.from_dict(pedidos,orient='index')
df_ped = df_ped.round(2) # Para eliminar dízimas periódicas

Data frame de pedidos criado. Vamos dar uma olhada nele.

In [42]:
df_ped.head()

,data_pedido,hora_pedido,id_cliente,bairro_pedido,cidade_pedido,cep_pedido,tipo_pagamento,agente,i_agua de coco,qt_agua de coco,...,vl_tot_pimentao [ 500g ],i_abobrinha,qt_abobrinha,vl_tot_abobrinha,i_quiabo [ kg ],qt_quiabo [ kg ],vl_tot_quiabo [ kg ],i_pepino amarelo [ unidade ],qt_pepino amarelo [ unidade ],vl_tot_pepino amarelo [ unidade ]
54,2020-07-03,11:30:53,39,aldeota,fortaleza,60150060,Pagar ao retirar na loja,android,True,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,2020-07-03,11:32:27,38,sao gerardo,fortaleza,60320105,Cartão de Crédito,windows,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,2020-07-03,11:38:01,35,manoel satiro,fortaleza,60713480,Cartão de Crédito,windows,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,2020-07-03,11:38:27,33,rodolfo teofilo,fortaleza,60431075,Débito/Transferência,android,True,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,2020-07-03,11:49:13,40,rodolfo teofilo,fortaleza,60430660,Cartão de Crédito,android,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


É possível perceber que ficamos com muitos valores nulos nesta matriz esparsa de items de cada pedido. Vamos verificar a quantidade de valores nulos por coluna.

In [43]:
pd.set_option("display.max_rows", 540)
df_ped.isnull().sum()

data_pedido                                                                                                      0
hora_pedido                                                                                                      0
id_cliente                                                                                                       0
bairro_pedido                                                                                                    0
cidade_pedido                                                                                                    0
cep_pedido                                                                                                       0
tipo_pagamento                                                                                                   0
agente                                                                                                           0
i_agua de coco                                                                  

In [44]:
pd.set_option("display.max_rows", 10)

Nas colunas de quantidade e preço é preciso trocar nulo por zero; nas colunas de produto nulo por False.

In [45]:
# Preenchendo as colunas numéricas com 0
for nome_col in df_ped.columns:
    if ('qt_' in nome_col) or ('vl_' in nome_col):
        df_ped[nome_col].fillna(0,inplace=True)

# Restaram somente as colunas de produtos, portanto podemos colocar False em tudo que estiver NaN DataFrame.
df_ped.fillna(False,inplace=True)

Vejamos como ficou o DataFrame de pedidos.

In [46]:
df_ped.head()

,data_pedido,hora_pedido,id_cliente,bairro_pedido,cidade_pedido,cep_pedido,tipo_pagamento,agente,i_agua de coco,qt_agua de coco,...,vl_tot_pimentao [ 500g ],i_abobrinha,qt_abobrinha,vl_tot_abobrinha,i_quiabo [ kg ],qt_quiabo [ kg ],vl_tot_quiabo [ kg ],i_pepino amarelo [ unidade ],qt_pepino amarelo [ unidade ],vl_tot_pepino amarelo [ unidade ]
54,2020-07-03,11:30:53,39,aldeota,fortaleza,60150060,Pagar ao retirar na loja,android,True,2.0,...,0.0,False,0.0,0.0,False,0.0,0.0,False,0.0,0.0
56,2020-07-03,11:32:27,38,sao gerardo,fortaleza,60320105,Cartão de Crédito,windows,False,0.0,...,0.0,False,0.0,0.0,False,0.0,0.0,False,0.0,0.0
59,2020-07-03,11:38:01,35,manoel satiro,fortaleza,60713480,Cartão de Crédito,windows,False,0.0,...,0.0,False,0.0,0.0,False,0.0,0.0,False,0.0,0.0
60,2020-07-03,11:38:27,33,rodolfo teofilo,fortaleza,60431075,Débito/Transferência,android,True,2.0,...,0.0,False,0.0,0.0,False,0.0,0.0,False,0.0,0.0
64,2020-07-03,11:49:13,40,rodolfo teofilo,fortaleza,60430660,Cartão de Crédito,android,False,0.0,...,0.0,False,0.0,0.0,False,0.0,0.0,False,0.0,0.0


## Criação de DataFrame de clientes.

Há análises que podem ser feitas baseando-se no perfil dos clientes, e não nos pedidos. Vamos criar um DataFrame com uma linha para cada cliente, totalizando valores e quantidades de itens de todas as compras.

In [47]:
# Esta função troca os nomes das colunas totalizadoras de valores e itens de forma a ficarem mais condizentes com a
# proposta do DataFrame.
def troca_nome_cols_tot(coluna):
    if coluna == 'vl_final_pedido':
        return 'vl_total_compras'
    elif coluna == 'qt_final_itens':
        return 'qt_total_itens'
    else:
        return coluna

cols_interesse = list()

for coluna in df_ped.columns:
    if 'vl_' in coluna or 'qt_' in coluna:
        cols_interesse.append(coluna)

dic_cols_interesse = dict()
for col_interesse in cols_interesse:
    dic_cols_interesse[col_interesse] = ['sum']

df_cli = df_ped.groupby('id_cliente').agg(dic_cols_interesse)
df_cli.columns = [col[0] for col in df_cli.columns] # Para índice deixar de ser tuplas com  nome do campo e 'sum'.
df_cli.columns = [troca_nome_cols_tot(coluna) for coluna in df_cli.columns]

# Acrescentando a coluna com o total de pedidos
qt_pedidos = df_ped.groupby('id_cliente').size()
qt_pedidos.name = 'qt_tot_pedidos'
df_cli = df_cli.join(qt_pedidos)
df_cli = df_cli.round(2) # Para eliminar dízimas periódicas

Vamos ver como ficou o DataFrame de clientes.

In [48]:
df_cli.head()

,qt_agua de coco,vl_tot_agua de coco,vl_total_compras,qt_total_itens,qt_cajuina,vl_tot_cajuina,qt_couve,vl_tot_couve,qt_doce de leite,vl_tot_doce de leite,...,vl_tot_cheiro verde [ maco ],qt_pimentao [ 500g ],vl_tot_pimentao [ 500g ],qt_abobrinha,vl_tot_abobrinha,qt_quiabo [ kg ],vl_tot_quiabo [ kg ],qt_pepino amarelo [ unidade ],vl_tot_pepino amarelo [ unidade ],qt_tot_pedidos
id_cliente,,,,,,,,,,,,,,,,,,,,,
31,32.0,96.0,2109.05,214,6.0,60.0,6.0,18.0,2.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12
33,2.0,6.0,290.80,34,0.0,0.0,4.0,12.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
34,61.0,183.0,9108.60,882,7.0,70.0,58.0,174.0,7.0,84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12
35,0.0,0.0,774.05,69,1.0,10.0,0.0,0.0,1.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
36,0.0,0.0,190.80,32,0.0,0.0,6.0,18.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


## Salvando os DataFrames para uso posterior

In [49]:
df.to_csv('arquivos/pedidos_long_final.csv',index=False)
df_ped.to_csv('arquivos/pedidos_wide_final.csv',index_label='id_pedido')
df_cli.to_csv('arquivos/clientes_final.csv')